In [3]:
from langchain_community.document_loaders import YoutubeLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [4]:
urls = ["https://www.youtube.com/watch?v=82QusHUE9jk","https://www.youtube.com/watch?v=rU63Mpr5-Q4","https://www.youtube.com/watch?v=EtsbdwVE_1s","https://www.youtube.com/watch?v=iQJ7EPxvTLY","https://www.youtube.com/watch?v=O8K2UEZzhBo"]

combined_content = []
for url in urls:
    loader = YoutubeLoader.from_youtube_url(f"{url}", add_video_info=False)
    x = loader.load()
    for item in x :
        combined_content.append(item)

In [6]:
combined_content

[Document(metadata={'source': '82QusHUE9jk'}, page_content="the government in Bangladesh has imposed a nationwide curfew and called on the Army to help quell protests in which at least 67 people have been killed students have been demanding that ministers scrap a quota system which reserves many public sector jobs for the relatives of veterans of the country's war of independence the UN has said violence against the protesters has been shocking and unacceptable and a spokesperson for the Bangladesh police has said around 300 Bangladeshi police officers were injured during Friday clashes with demonstrators in multiple locations around the capital DCA let's now speak to Nami Hussein who is a professor of development studies at the school of Oriental and African studies welcome to the program could you explain a little bit if you would the background to these process and why there is a divide really over the legacy of the country's history well thank you for covering this story as you as 

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 500)
final_documents = text_splitter.split_documents(combined_content)

In [8]:
embeddings = HuggingFaceEmbeddings(model_name = "hkunlp/instructor-large")

c:\Users\s\miniconda3\envs\langchainenv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\s\miniconda3\envs\langchainenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
db = FAISS.from_documents(final_documents,embeddings)

In [10]:
db.save_local('faiss_index')

In [12]:
new_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)

In [13]:
import os 
from langchain_groq import ChatGroq
groq_api_key = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model='llama3-8b-8192', groq_api_key= groq_api_key,temperature=0)

In [22]:
### USe of Retrieval Chain 
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

prompt = ChatPromptTemplate.from_template(
"""
You are a helpful assitant.Answer The following Question based on the context :
<context>
{context}
</context>

And only give show the translated  output into the {language}

"""
)


# prompt = ChatPromptTemplate.from_messages(
#     [
#         ('system',"You are a helpful assistant .Answer The question to the nest of your ability"),
#         MessagesPlaceholder(variable_name="message")
#     ]
# )


parser = StrOutputParser()
document_chain = create_stuff_documents_chain(llm , prompt,output_parser=parser)

# Create a retriever from the vector store
retriever = new_db.as_retriever()


# Create a retrieval chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Define the input query
query = "why student are  protesting  in bangladesh?"

# Get the response from the retrieval chain
#
response = retrieval_chain.invoke({'input': query,'language':'Bangla'})
  #return response['answer']
#translation_into('Bangla')
# Print the answer
print(response['answer'])



Here is the translated output in Bangla:

বাংলাদেশে সরকারি বিশ্ববিদ্যালয় ক্যাম্পাসগুলি অনির্দিষ্টকালের জন্য বন্ধ করা হয়েছে জুলাই ১৭ তারিখ থেকে। এটা প্রতিবাদী আন্দোলনের ফলে হয়েছে। এই প্রতিবাদীরা কোডা সিস্টেম রিফর্ম চায়। কোডা সিস্টেমটি ১৯৭১ সালে দেশের স্বাধীনতা অর্জনের পর থেকে চালু আছে। এই সিস্টেমটি মুক্তিযুদ্ধের সময় সংগ্রামী বাহিনীর সদস্যদের পরিবারের সদস্যদের জন্য ৩০% চাকরি বরাদ্দ করে। কিন্তু ছাত্ররা এই সিস্টেমটি বদলাতে চায় কারণ এই সিস্টেমটি মেরিটেড এবং প্রতিভাবান ছাত্রদের জন্য সুযোগ কমিয়ে দেয়।

এই প্রতিবাদীদের অভিযোগ হল যে এই সিস্টেমটি রাজনৈতিক দলের সদস্যদের জন্য আরেকটা সুযোগ দেয়। প্রধানমন্ত্রী এই প্রতিবাদীদের উপর কটুক্তি করেছেন যে এই প্রতিবাদীরা রাকাসদের সন্তান। এই কটুক্তি প্রতিবাদীদের আরও ক্ষোভ বাড়িয়ে দিয়েছে।


In [20]:
store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]  



config = {"configurable":{"session_id":"chat1"}}

with_message_history = RunnableWithMessageHistory(llm,get_session_history)


response2 = with_message_history.invoke(
    [
        HumanMessage(content=f"{response['answer']}"),
    ],
    config=config
)


In [21]:
response

{'input': 'why student protest in bangladesh?',
 'language': 'Bangla',
 'context': [Document(metadata={'source': 'EtsbdwVE_1s'}, page_content="at people from the helicopters they're shooting rounds of tear gases wow oh my gosh so let's go back to when July 17th that's when Bangladesh started shutting down Public University campuses indefinitely right as a response to these protests talk to me about the protesters who are these students who are leading the demonstrations and what are they calling for the students their ask was a Reformation of the Koda system which has been in place for a long long time since they were independent and the Cota system it gives 30 % government jobs to relatives and families of the freedom fighters who fought in the Liberation war of 1971 and their ask is since it's been 53 years Bangladesh is liberated families of Freedom Fighters have already grown up enough so the jobs which are offered to the grandchildren or great grandchildren of the Freedom Fighters

In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]  



config = {"configurable":{"session_id":"chat1"}}

with_message_history = RunnableWithMessageHistory(llm,get_session_history)


response = with_message_history.invoke(
    [
        HumanMessage(content='My name is Arif'),
    ],
    config=config
)


In [16]:
response

AIMessage(content="Assalamu alaikum, Arif! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 15, 'total_tokens': 49, 'completion_time': 0.027125222, 'prompt_time': 0.004385468, 'queue_time': None, 'total_time': 0.03151069}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-cfa0e737-6461-43bd-9563-76aa53a83594-0', usage_metadata={'input_tokens': 15, 'output_tokens': 34, 'total_tokens': 49})